## Алгоритм k-means

В этом блокноте приводится реализация алгоритма k-means и кластеризация пользователей относительно оценок, которые они дали фильмам, с целью выделить группы похожих пользователей

In [1]:
import pandas as pd
import os
import numpy as np

DATA_PATH = '../data/'

ratings_main = pd.read_csv(
    os.path.join(DATA_PATH, 'ratings.csv'),
    dtype={
        'user_uid': np.uint32,
        'element_uid': np.uint16,
        'rating': np.uint8,
        'ts': np.float64,
    }
)

In [2]:
ratings_main.head(6)

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07
5,557663,1918,10,4.430505e+07


In [3]:
#вырежем первые 2000 строк для тестирования
ratings = ratings_main.iloc[:2000]

In [4]:
movies_list = list(set(ratings['element_uid'])) 
#gets the list of all movies that have been rated by at least someone

In [5]:
def generate_ratings_dict(data) -> dict:
    ratings_dict = {}
    
    for _, item in data.iterrows():
        user_id = item['user_uid']
        
        if user_id not in ratings_dict:
            ratings_dict[user_id] = {}
        
        (ratings_dict[user_id])[item['element_uid']] = item['rating']
    
    return ratings_dict

In [6]:
ratings_dict = generate_ratings_dict(ratings)

In [7]:
def generate_data(ratings_dict: dict, movies_list: list) -> dict:
    data_dict = {}
    n = len(movies_list)
    
    for user_id, item in ratings_dict.items():        
        if user_id not in data_dict:
            data_dict[user_id] = [0]*n
        
        for i in range(n):
            if movies_list[i] in ratings_dict[user_id]:
                (data_dict[user_id])[i] = (ratings_dict[user_id])[movies_list[i]]
    
    return data_dict

In [8]:
data_dict = generate_data(ratings_dict, movies_list)

In [9]:
import random

def generate_centroids(k: int, data_list: list) -> list: #chooses random k users as initial centroids
    return random.sample(data_list, k)

In [10]:
generate_centroids(4, list(data_dict))

[467946.0, 375252.0, 442550.0, 345066.0]

In [11]:
import math

def distance(u: list, v: list) -> float: #finds the distance between two vectors
    dist = 0

    for i in range(len(u)):
        dist += math.pow((u[i] - v[i]), 2)

    return math.sqrt(dist)

In [12]:
def add_to_cluster(item: int, centroids: list, data_dict: dict): #centroids = list of lists
    return item, min(range(len(centroids)), key = lambda i: distance(data_dict[item], centroids[i]))

In [13]:
from functools import reduce

def add_vector(u: list, v: list) -> list:
    return [u[k] + v[k] for k in range(len(v))]

def move_centroids(k: int, iteration: list, data_dict: dict) -> list:
    centroids = []

    for cen in range(k):
        members = [i[0] for i in iteration if i[1] == cen] #finds all members of this cluster

        if members:
            centroid = [i/len(members) for i in reduce(add_vector, [data_dict[m] for m in members])]
            centroids.append(centroid)
    
    return centroids

In [14]:
def k_means(k: int, data_dict: dict) -> list:
    best_weight = math.inf
    data_list = list(data_dict)
    centroids = [data_dict[i] for i in generate_centroids(k, data_list)]

    while True:
        iteration = list([add_to_cluster(item, centroids, data_dict) for item in data_list])
        new_weight = 0
        
        for i in iteration:
            new_weight += distance(data_dict[i[0]], centroids[i[1]]) 
            #calculates the distance between each item and its centroid
        
        #if the new weight is better than the best weight, it continues;
        #otherwise, it returns
        if new_weight < best_weight:
            best_weight = new_weight
            new_weight = 0
        else:
            return iteration

        centroids = move_centroids(k, iteration, data_dict)

In [15]:
clustered_users = k_means(4, data_dict)

In [16]:
sorted(clustered_users, key = lambda t : t[0])[:10]

[(504.0, 0),
 (1120.0, 0),
 (1464.0, 0),
 (1671.0, 0),
 (1942.0, 0),
 (3074.0, 3),
 (4539.0, 0),
 (5113.0, 0),
 (5177.0, 0),
 (5988.0, 0)]

Проделаем то же с помощью библиотеки ```scikit-learn``` и сравним результаты:

In [ ]:
X = list(data_dict.values())

from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=4, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [ ]:
clustered_sklearn = [(list(data_dict)[i], y_km[i]) for i in range(len(y_km))]
sorted(clustered_sklearn, key = lambda t : t[0])[:10]

In [ ]:
symm_diff = set()
intersection = set()
clustered_users = sorted(clustered_users, key = lambda t : t[0])
clustered_sklearn = sorted(clustered_sklearn, key = lambda t : t[0])

#так как кластеры могут нумероваться в любом порядке, сравним кластеры, 
#соответствующие первому пользователю в отсортированном списке
num_1 = clustered_users[0][1]
num_2 = clustered_sklearn[0][1]

for i in range(len(clustered_users)):
    if (clustered_users[i][1] == num_1 and clustered_sklearn[i][1] != num_2) or (clustered_users[i][1] != num_1 and clustered_sklearn[i][1] == num_2):
        symm_diff.add(clustered_users[i])
    elif clustered_users[i][1] == num_1 and clustered_sklearn[i][1] == num_2:
        intersection.add(clustered_users[i])

format(100*len(symm_diff)/len(intersection), '.2f') 
#выводится отношение (в %) симметрической разности множеств пользователей для этих двух кластеров к их пересечению

Заметим, что полученное значение мало, что говорит о корректности реализации алгоритма